In [1]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np
import matplotlib.pyplot as plt
from datasets.rsna_breast_cancer import BreastCancerDataset, DatasetSplit
import backbones
from simplenet import SimpleNet
import utils
from torch.utils.data import DataLoader
import os

C:\Users\Max Ksoll\Documents\Git\SimpleNet\venv\lib\site-packages\torchvision\transforms\functional_pil.py:5: UserWarning: The torchvision.transforms.functional_pil module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(


In [2]:
img_dir = "data/rsna_breast_cancer/rsna_breast_cancer"
csv_file = "data/rsna_breast_cancer/train.csv"

ds = BreastCancerDataset(
    img_dir=img_dir,
    meta_data_csv_path=csv_file,
    train_val_test_split=(0.9, 0.1, 0)
)

val_ds = BreastCancerDataset(
    img_dir=img_dir,
    meta_data_csv_path=csv_file,
    split=DatasetSplit.VAL,
    train_val_test_split=(0.9, 0.1, 0)
)
train_loader = DataLoader(ds, batch_size=40, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=40, shuffle=True)

In [3]:
backbone = backbones.load('efficientnet_b1')

device = 'cuda'

net = SimpleNet(device)
net.load(
        backbone=backbone,
        layers_to_extract_from=['conv_head'],
        device=device,
        input_shape=(3,224,224),
        pretrain_embed_dimension=64,
        target_embed_dimension=64,      # Projection dimensionality
        patchsize=3,                    # Größe von Patches, in die die Features geteilt werden
        embedding_size=None,            # unused Parameter
        meta_epochs=20,                 # Overall Epochs
        aed_meta_epochs=0,              # used for cos_lr scheduler, but needs to be an int allways
        gan_epochs=2,                   # Discriminator Training Epochs per Epoch
        noise_std=0.05,                 # Stärke des Noise für die Fake Bilder
        dsc_layers=2,
        dsc_hidden=32,
        dsc_margin=0.5,
        dsc_lr=0.0001,
        auto_noise=0,                   # scheint ein sinnloser Parameter zu sein
        train_backbone=True,
        cos_lr=False,
        pre_proj=0,                     # Number of Layers for Projection
        proj_layer_type=0,              # if > 1 then relu is added to all but the last layer of Projection
        mix_noise=1,                    # ich hab keine Ahnung
    )
models_dir = "models"
dataset_name = "rsna_breast_cancer"
net.set_model_dir(models_dir, dataset_name)

In [4]:
i_auroc, p_auroc, pro_auroc = net.train(train_loader, val_loader)
print(i_auroc, pro_auroc, pro_auroc)

epoch:1 loss:1.0 lr:0.0001 p_true:0.0 p_fake:0.0: 100%|██████████| 2/2 [19:34<00:00, 587.11s/it]        


----- 0 I-AUROC:0.4974(MAX:0.4974)  P-AUROC-1(MAX:-1) -----  PRO-AUROC-1(MAX:-1) -----


epoch:1 loss:1.0 lr:0.0001 p_true:0.0 p_fake:0.0: 100%|██████████| 2/2 [10:26<00:00, 313.45s/it]


----- 1 I-AUROC:0.5055(MAX:0.5055)  P-AUROC-1(MAX:-1) -----  PRO-AUROC-1(MAX:-1) -----


epoch:1 loss:1.0 lr:0.0001 p_true:0.0 p_fake:0.0: 100%|██████████| 2/2 [10:28<00:00, 314.20s/it]


...

----- 17 I-AUROC:0.5078(MAX:0.5398)  P-AUROC-1(MAX:-1) -----  PRO-AUROC-1(MAX:-1) -----


epoch:1 loss:1.0 lr:0.0001 p_true:0.0 p_fake:0.0: 100%|██████████| 2/2 [09:55<00:00, 297.74s/it]


----- 18 I-AUROC:0.5281(MAX:0.5398)  P-AUROC-1(MAX:-1) -----  PRO-AUROC-1(MAX:-1) -----


epoch:1 loss:1.0 lr:0.0001 p_true:0.0 p_fake:0.0: 100%|██████████| 2/2 [17:13<00:00, 516.78s/it]


----- 19 I-AUROC:0.529(MAX:0.5398)  P-AUROC-1(MAX:-1) -----  PRO-AUROC-1(MAX:-1) -----
0.5398357756334276 -1 -1


In [5]:
res = net.predict(val_loader)

In [7]:
scores, masks, features, labels_gt, masks_gt = res

In [11]:
sum(scores)/len(scores)

24.23278204689947